 <table><tr><td with='5'></td># Implementing logistic Regression from Scratch</td></tr>
 <tr><td background-color:'gray'></td><td>Created on Fri Apr 15 12:02:15 2016</td></tr>
 <tr><td></td><td>
 
 The goal of this assignment is to implement your own logistic regression classifier.
<ol>
    <li> extract features from Amazon product Reviews
    <li> Convert dataFrame into a Numpy array
    <li> Implement the link function for logistic regression
    <li> Write a function to compute the derivative of the log likelihood function with respect to a single coefficient.
    <li> implement gradient ascent
    <li> Given a set of coefficients, predict sentiments
    <li> Compute classification accuracy for the logistic regression model
</ol>
  
 </td></tr></table>


In [32]:
# -*- coding: utf-8 -*-
"""


@author: jmlbeaujour@gmail.com
"""

import pandas as pd
import string
import json
import numpy as np
import matplotlib.pyplot as plt

## Functions:
<ul>
 <li> remove_punctuation(text)
 Takes a line of text and removes all punctuation
 <li> dataFrame2Array(dataframe, features, label)
 return 2 arrays: 2D array feature_matrix, label_array i.e real sentiment
 <li> predict_probability(feature_matrix, coefficients)
 <li> feature_derivative(errors, feature)
 <li> compute_log_likelihood(feature_matrix, sentiment, coefficients)
 <li> logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter)
</ul>

In [33]:
def remove_punctuation(text):
    return text.translate(None, string.punctuation)   

def dataFrame2Array(dataframe, features, label):
    dataframe['constant'] = 1 #add the bias term
    #add 'contains_' to all features to match column name #features is a list of string
    features = ['constant']+['contains_'+x for x in features] 
    feature_matrix = np.array(dataframe[features]) #extract set of columns from dataframe and convert to matrix
    label_array = np.array(dataframe[label]) #create an array with the label (actual value)
    label_array = np.reshape(label_array, (np.shape(label_array)[0],1))
    return (feature_matrix, label_array)

### Estimate conditional probability with link function:
The link function is given by:
\begin{align}
P(y^{(i)}) = +1 | x^{(i)}, w) = \frac{1}{1 + exp(-w^T x^{(i)})}
\end{align}
where the feature vector $x^{(i)}$ represents the word counts of important_words in the review $x^{(i)}$.
$P(y=+1|x,w)$ is the probability that sentiment of example 1 is 1 parametrized by  $w$:
    
\begin{align}
P(y=+1|x,w) = sigmoid(w^T x) = \frac{1}{(1+exp(-w^T x))}
\end{align}

Since the word counts are stored as columns in [feature\_matrix], each i-th row of the matrix corresponds to the feature vector $x^{(i)}$:
\begin{align}
[feature\_matrix] = \left[\begin{matrix} (x^{(1)})^T \\(x^{(2)})^T\\.\\.\\.\\(x^{(m)})^T\end{matrix} \right] = \left[\begin{matrix} x_0^{(1)} & x_1^{(1)} & x_2^{(1)} & .... & x_n^{(1)} \\
x_0^{(2)} & x_1^{(2)} & x_2^{(2)} & .... & x_n^{(2)} \\
.&.&.&.&.\\
.&.&.&.&.\\
x_0^{(m)} & x_1^{(m)} & x_2^{(m)} & .... & x_n^{(m)}
\end{matrix} \right]
\end{align}
The score vector is defined by:
\begin{align}
[score] = [feature\_matrix] * w = \left[\begin{matrix} (x^{(1)})^T \\(x^{(2)})^T\\.\\.\\.\\(x^{(m)})^T\end{matrix} \right] * \left[ \begin{matrix} w_0 \\w_1\\w_2\\.\\.\\.\\w_n \end{matrix} \right] = \left[\begin{matrix} x_0^{(1)} & x_1^{(1)} & x_2^{(1)} & .... & x_n^{(1)} \\
x_0^{(2)} & x_1^{(2)} & x_2^{(2)} & .... & x_n^{(2)} \\
.&.&.&.&.\\
.&.&.&.&.\\
x_0^{(m)} & x_1^{(m)} & x_2^{(m)} & .... & x_n^{(m)}
\end{matrix} \right] * \left[ \begin{matrix} w_0 \\w_1\\w_2\\.\\.\\.\\w_n \end{matrix} \right]
\end{align}

\begin{align}
[score] = \left[\begin{matrix} x_0^{(1)}*w_0 + x_1^{(1)} * w_1 + x_2^{(1)}*w_2 + .... + x_n^{(1)}*w_n \\
x_0^{(2)}*w_0 + x_1^{(2)} * w_1 + x_2^{(2)}*w_2 + .... + x_n^{(2)}*w_n \\
x_0^{(3)}*w_0 + x_1^{(3)} * w_1 + x_2^{(3)}*w_2 + .... + x_n^{(3)}*w_n \\
.\\
.\\
.\\
x_0^{(m)}*w_0 + x_1^{(m)} * w_1 + x_2^{(m)}*w_2 + .... + x_n^{(m)}*w_n \\
\end{matrix} \right]
\end{align}


In [34]:
def predict_probability(feature_matrix, coefficients):
    score = np.dot(feature_matrix, coefficients)
    predictions = 1/(1 + np.exp(-score))
    return predictions


### Compute derivative of log-likelihood with respect to a single coefficient $w_j$:

\begin{align}
\frac{\partial \ell}{ \partial w_j} = \sum_{i=1} ^m x_j^{(i)} * [errors]
\end{align}

where 
\begin{align}
[errors] = \left(\mathbb{1}[y^{(i)}=+1] - P(y^{(i)}=+1 | x^{(i)},w) \right)
\end{align}
The log likelihood simplifies the derivation of the gradient and is more numerically stable. 
\begin{align}
\ell \ell (w) = \sum _{i=1} ^m \left( (\mathbb{1}[y^{(i)}=+1]-1)w^{T} x^{(i)} - ln(1+exp(-w^{T} x^{(i)})) \right)
\end{align}


In [35]:
def feature_derivative(errors, feature):
    derivative = np.dot(np.transpose(feature), errors)
    return derivative

def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment == +1)    
    scores = np.dot(feature_matrix, coefficients)
    lp = np.sum((indicator - 1)*scores -np.log(1. + np.exp(-scores)))
    return lp

### logictic_regression() takes gradient steps toward optimum.
The function accepts the following parameters:
<ol>
<li> [feature_matrix]: 2D array of features
<li> [sentiment] : 1D array of class labels
<li> [initial_coefficients]: 1D array containing initial values of coefficients
<li> [step_size]: a parameter controlling the size of the gradient steps
<li> [max_iter] : number of iterations to run gradient ascent.
</ol>
and returns the last set of coefficients after performing gradient ascent


In [36]:
def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients)
    for itr in xrange(max_iter):
        predictions = predict_probability(feature_matrix, coefficients)
        indicator = (sentiment == +1)
        errors = indicator - predictions
        for j in xrange(len(coefficients)): #loop over each coefficient
            derivative = feature_derivative(errors, feature_matrix[:,j])
            coefficients[j] = coefficients[j] + step_size * derivative
        
        if itr<=15 or (itr <= 100 and itr%10 == 0) or (itr <= 1000 and itr%100 == 0) or (itr <= 10000 and itr%100 == 0) \
        or itr%10000 == 0:
            lp = compute_log_likelihood(feature_matrix, sentiment, coefficients)
            print 'iteration %*d: log likelihood of observed labels = %.8f' %(int(np.ceil(np.log10(max_iter))), itr, lp)
    return coefficients 

# Load review dataset in dataFrame. 

<ol> 
<li> IMPORT CSV FILE (TRAINING SET):
The csv file has 4 cols:
<ul>
    <li> name of product = name,
    <li> review (type='text'),
    <li> rating (type=int $\in$ {0,1,2,3,4,5}),
    <li> sentiment (type=in $\in$ {1(positive sentiment), -1(negative sentiment)}
</ul>

<li> APPLY TEXT CLEANING ON ALL REVIEWS: 
remove the punctuations and fills n/a values with empty string: ''
<li> IMPORT important_words.json file:
contains a list of 193 words most frequently used. This will be our feature.
</ol>

In [37]:
dataFile = r'E:/DataScientist/myNotebook/ML_classification (Uni.Washington)/assig2/amazon_baby_subset.csv'
file_important_words = r'E:/DataScientist/myNotebook/ML_classification (Uni.Washington)/assig2/important_words.json'
important_words = json.load(open(file_important_words)) #open the json file as a string and parse it with json.load ==> a list
nbr_features = len(important_words)
print 'nbr of features:', nbr_features
colNames = ['name', 'review', 'rating', 'sentiment']
#products = pd.read_table(dataFile, sep=' ', header=None, names=colNames) # user_id gender  age  occupation    zip
products = pd.read_csv(dataFile, header=0, names=colNames) #[shape=(183531,3)].\n"
#load important_word json file
print products['review'][0]
#Name of the first 10 products
print 'List of the first 10 products:', products['name'][1:11]  #must use iloc to return element at index id (products.iloc[1])

print 'Generate review_clean column'
#for the empty review, fill (n/a)
products = products.fillna({'review':''})
#Apply text cleaning to the data: create a new column with review without punctuations
products['review_clean'] = products['review'].apply(remove_punctuation)

nbr of features: 193
All of my kids have cried non-stop when I tried to ween them off their pacifier, until I found Thumbuddy To Love's Binky Fairy Puppet.  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from it.This is a must buy book, and a great gift for expecting parents!!  You will save them soo many headaches.Thanks for this book!  You all rock!!
List of the first 10 products: 1       Nature's Lullabies Second Year Sticker Calendar
2       Nature's Lullabies Second Year Sticker Calendar
3                           Lamaze Peekaboo, I Love You
4     SoftPlay Peek-A-Boo Where's Elmo A Children's ...
5                             Our Baby Girl Memory Book
6     Hunnt&reg; Falling Flowers and Birds Kids Nurs...
7     Blessed By Pope Benedict XVI Divine Mercy Full...
8     Cloth Diaper Pins Stainless Steel Traditional ...
9     Cloth Diaper Pins Stainless Steel Traditional ...
10    Newborn Baby Tracker&reg; - Round

## Generate feature columns for each review/training example

<ol>
<li> For each word in important_words (193 words), the number of times the word occurs in the review is reported in a column: 'contains_important_words[j]'.
<li> print the number of product reviews that contain the word perfect
</ol>

Notes:
<ul>
<li> list.count(obj) counts occurence of obj in list
<li> Python supports the creation of anonymous functions (i.e. functions that are not bound to a name) at runtime, using a construct called "lambda". The lambda definition does not include a "return" statement.
For example: #def f (x): return x**2 is equivalent to g = lambda x: x**2


In [38]:
print 'Generate columns w/ count in review of important_words'
for word in important_words:
    products['contains_'+word] = products['review_clean'].apply(lambda s: s.split().count(word))
#print 'this is product 0:', products.iloc[0]

print 'Number of reviews with the word perfect :', products[products['contains_perfect']>0].shape[0]

Generate columns w/ count in review of important_words
Number of reviews with the word perfect : 2955


### Generate feature_matrix, sentiment array
<ol>
<li> Convert the dataFrame to a multidimentional array
<li> initialize the coefficients $\vec{w}$
<li> print the number of features (including bias)
</ol>

In [39]:
dataFrame_arrays = dataFrame2Array(products, important_words, 'sentiment')
feature_matrix = dataFrame_arrays[0] #item #9
Nbr_of_examples = np.shape(feature_matrix)[0]
sentiment = dataFrame_arrays[1]
initial_coefficients = np.zeros((nbr_features+1,1))
step_size = 1e-7
max_iter = 301
print 'The feature_matrix has < ', np.shape(feature_matrix)[1],' > features, including the bias/intercept'


The feature_matrix has <  194  > features, including the bias/intercept


### Run learning algorithm (logistic_regression)

<ul>
<li> The function [logistic_regression()] carries out the following steps:
<ol>
<li> initialize vector [coefficients] to [initial_coefficients]
<li> Predict the class probability $P(y^{(i)}=+1|x^{(i)},w)$ using [predict_probability()] and save it to the variable [predictions]
<li> Compute indicator value for $(y^{(i)}=+1)$ by comparing [sentiment] against +1. Save it to a variable [indicator].
<li> Compute the errors as difference between [indicator] and [predictions], and save it to variable [errors]
<li> For each $j-$th coeffcient, compute the per-coefficient derivative by calling [feature_derivative] with the $j-$th column of [feature_matrix]. Then increment the $j-$th coefficient by ([step_size]*[derivative]
</ol>

<li> Predicting sentiments
<ol> 
<li> Class predictions for a data $x$ can be computed from the coefficients $w$ using the following formula:
\begin{align}
y = 
\begin{cases}
    +1 & x^T w \geq 0 \\
    -1 & x^T w \leq 0
  \end{cases}
\end{align}
<li> first compute the [scores] using [feature_matrix] and [coefficients] and a dot product
<li> apply the threshold 0 on the [scores] to compute the class predictions.
</ol>
<li> Measure the accuracy of the algorithm:

\begin{align}
accuracy = \frac{\text{# correctly classified data points}}{\text{# total data points}}
\end{align}
</ul>

In [40]:
coefficients = logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter)
score = np.dot(feature_matrix, coefficients)
score[score > 0] = 1
score[score <= 0] = -1
#Number of product reviews that contain the words perfect
comparison = np.zeros((Nbr_of_examples,1))
comparison = (sentiment==score)
print 'How many reviews were predicted to have positive sentiment', sum(score > 0)
print sum(comparison==1)
accuracy = float(sum(comparison==1))/Nbr_of_examples*100
print 'What is the accuracy of the model on predictions made above: ', accuracy

iteration   0: log likelihood of observed labels = -36780.91768478
iteration   1: log likelihood of observed labels = -36775.13434712
iteration   2: log likelihood of observed labels = -36769.35713564
iteration   3: log likelihood of observed labels = -36763.58603240
iteration   4: log likelihood of observed labels = -36757.82101962
iteration   5: log likelihood of observed labels = -36752.06207964
iteration   6: log likelihood of observed labels = -36746.30919497
iteration   7: log likelihood of observed labels = -36740.56234821
iteration   8: log likelihood of observed labels = -36734.82152213
iteration   9: log likelihood of observed labels = -36729.08669961
iteration  10: log likelihood of observed labels = -36723.35786366
iteration  11: log likelihood of observed labels = -36717.63499744
iteration  12: log likelihood of observed labels = -36711.91808422
iteration  13: log likelihood of observed labels = -36706.20710739
iteration  14: log likelihood of observed labels = -36700.5020

### Result analysis: words contributing more to the positive and negative reviews
<ul>
<li> create a tuple (word,coefficient_value) 
    <ul>
        <li> coefficients = list(coefficients[1:]) and,
        <li> word[ : ]
        <li> zip
    </ul>
</ul>

In [41]:
coefficients = coefficients.tolist()[1:]
word_coefficient_tuple = sorted(zip(coefficients, important_words), key=lambda x:x[0], reverse=True)
print 'The 10 words that have the most positive coeffcient values:', word_coefficient_tuple[0:10]

word_coefficient_tuple = sorted(zip(coefficients, important_words), key=lambda x:x[0])
print 'The 10 words that have the most negative coeffcient values:', word_coefficient_tuple[0:10]

The 10 words that have the most positive coeffcient values: [([0.0665460841704577], 'great'), ([0.06589076292212327], 'love'), ([0.0647945868025784], 'easy'), ([0.04543562630842137], 'little'), ([0.04497640139490604], 'loves'), ([0.030135001092107077], 'well'), ([0.02973993710496846], 'perfect'), ([0.020077541034775378], 'old'), ([0.018408707995268992], 'nice'), ([0.01770319990570169], 'daughter')]
The 10 words that have the most negative coeffcient values: [([-0.053860148445203135], 'would'), ([-0.04151103339210889], 'product'), ([-0.038982037286487116], 'money'), ([-0.03306951529475273], 'work'), ([-0.030051249236035804], 'even'), ([-0.028978976142317068], 'disappointed'), ([-0.028711552980192574], 'get'), ([-0.027742697230661334], 'back'), ([-0.026592778462247283], 'return'), ([-0.024482100545891717], 'monitor')]
